## Squid Marking Scheme Viewer ##

This notebook reads the variant number of the written answer question from a quiz paper (via the ArUco tag) and shows you the corresponding marking scheme. You'll need a webcam (ideally an external one on a little tripod to point at your desk) and you need to allow your browser to access your camera.

Click on "Upload marking scheme (.json)" and select the .json file that was emailed to you.

If the webcam can see the ArUco tag at the top of the quiz paper, then it will automatically display the marking scheme for that variant. It might take a second or two to read.



#### Installation ####
No installation required, this is meant to run in Binder. Just execute the following code cell.

Copyright (c) 2022 by Florian Breuer (florian.breuer@newcastle.edu.au) with special thanks to Marcus Chijoff.

In [1]:
# Built-in imports:
import io
import json
import os
import threading
from time import sleep

# External imports:
import cv2
from ipywebrtc import CameraStream, ImageRecorder
import ipywidgets as widgets
import numpy as np
import PIL.Image
# import PIL.ImageFilter

# Globals:
pool = []
num_variants = 0
quitnow = False
v = 0  # current variant number
    
# Set up ArUco detector
aruco_variant = cv2.aruco.DICT_5X5_100
aruco = cv2.aruco.Dictionary_get(aruco_variant)
arucoParams = cv2.aruco.DetectorParameters_create()

camera = CameraStream.facing_environment(audio=False)
image_recorder = ImageRecorder(stream=camera)
image_recorder.autosave = False
image_recorder.recording = True
    
# Define widgets:
File = widgets.FileUpload(accept='.json', 
                          multiple=False,
                          description='Upload marking scheme (.json)',
                          layout={'width' : '50%'}
                         )

VariantDisplay = widgets.Label(value='Tag number:')
VariantSelector = widgets.Dropdown(options=[('None', 0)], 
                                   value=0, 
                                   description='Variant')
SolutionDisplay = widgets.HTMLMath(value='')
RubricDisplay = widgets.HTMLMath(value=r'''<b>Grading Scheme:</b><br> 
Mark the grading bubble on the Answer Sheet
with the score appropriate to the student's solution:<br><br>
<ul>
    <li> <b>3 marks:</b> The student demonstrates a good understanding and obtains the correct answer. </li>
    <li> <b>2 marks:</b> The student demonstrates a good understanding of how to do the problem
                (some minor errors permitted). </li>
    <li> <b>1 mark:</b> The student demonstrates a partial understanding of how to do the problem.</li>
    <li> <b>0 marks:</b> The solution is missing, completely wrong, or shows insufficient steps to
                 convince you that they obtained the answer themselves. </li>
</ul>''')
CloseButton = widgets.Button(description='Quit')

# Functions:
def load_json(b):
    global pool
    global v
    global num_variants
    pool = json.loads(File.data[-1])
    num_variants = len(pool["L"])
    VariantSelector.options = [('None', 0)] + list((f'Variant {i}', i) for i in range(1, num_variants + 1))
    if v >= 0 and v <= num_variants:
        VariantSelector.value = v
        update_solution(0)
    else:
        VariantSelector.value = 0

def close_camera(b):
    '''close camera and quit'''
    global quitnow
    camera.close_all()  # this kills all widgets!
    quitnow = True
    
def blip(b):
    '''blip image recorder, causing it to grab a frame from camera'''
    image_recorder.recording = True  
    
def read_variant(b):
    '''Read variant number from ArUco tag'''
    global v
    im_in = PIL.Image.open(io.BytesIO(image_recorder.image.value))
    im_array = np.array(im_in)[...,:3] # no alpha
    (corners, ids, rejected) = cv2.aruco.detectMarkers(im_array, aruco, parameters=arucoParams)
    if ids is not None:
        v = int(ids[0])
        if v >= 0 and v <= num_variants:
            VariantSelector.value = v
            update_solution(0)
        else:
            VariantSelector.value = 0
        VariantDisplay.value = f'Tag number: {v}'

def update_solution(b):
    '''show solution to current variant'''
    if v != 0:
        Q = pool['L'][v-1]
        SolutionDisplay.value = f'<b>Question</b> (variant {Q["variant_number"]})<br> {Q["question_text"]}' +\
                        f'<br><b>Solution.</b><br> {Q["solution_text"]}'
        
def select_variant(b):
    '''user has selected a different variant'''
    global v
    v = VariantSelector.value
    VariantDisplay.value = f'Variant: {v}'
    update_solution(0)
    
# Bind widget behaviour
File.observe(load_json, 'value')
CloseButton.on_click(close_camera)
image_recorder.image.observe(read_variant, 'value')
VariantSelector.observe(select_variant, 'value')

# Display widgets
display(File)
display(camera)
display(widgets.HBox([VariantDisplay, VariantSelector]))
display(SolutionDisplay)
display(RubricDisplay)
display(CloseButton)

# Finally, set up a ticker that regularly blips the image_recorder
def ticker():
    global quitnow
    while not quitnow:
        blip(0)
        sleep(1.5)
    print('Good bye!')
thread = threading.Thread(target=ticker)
thread.start()



FileUpload(value={}, accept='.json', description='Upload marking scheme (.json)', layout=Layout(width='50%'))

CameraStream(constraints={'audio': False, 'video': {'facingMode': 'environment'}})

HTMLMath(value='')

HTMLMath(value="<b>Grading Scheme:</b><br> \nMark the grading bubble on the Answer Sheet\nwith the score appro…

Button(description='Quit', style=ButtonStyle())

Good bye!
